In [25]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

## To run in Colab

In [ ]:
# !pip install fastai==1.0.59
# !pip install -q torch==1.4.0 torchvision==0.5.0

In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive', force_remount=True)
# !ls "/content/gdrive/My Drive/CVbyDL/data/minicity"

In [26]:
from fastai import *
from fastai.vision import *
import os
from glob import glob
from utils.databunch import *
from utils.metrics import acc_cityscapes, mIOU
from models.UNet.unet import UNet
from utils.evaluate import save_to_eval

In [27]:
# base_dir = "/content/gdrive/My Drive/CVbyDL/data/minicity"
# test_dir = "/content/gdrive/My Drive/CVbyDL/data/cityspaces"
base_dir = "data/minicity"
test_dir = "data/cityspaces"

In [28]:
data = get_databunch(base_dir, size =(256,512) , bs=4)

In [29]:
metrics= [acc_cityscapes, mIOU]


## Prepare UNet model

In [30]:
learn = unet_learner(data,models.resnet34, metrics=metrics,pretrained=False)

In [31]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [40]:
lr = 1e-4
learn.fit_one_cycle(5, slice(lr), pct_start=0.9)

epoch,train_loss,valid_loss,acc_cityscapes,mIOU,time
0,1.246868,1.260979,0.710874,0.164820,00:30
1,1.223689,1.264411,0.707533,0.172013,00:31
2,1.244406,1.372356,0.699041,0.134432,00:31
3,1.253105,1.279647,0.703986,0.143871,00:31
4,1.214219,1.189472,0.720710,0.184479,00:31


In [33]:
learn.save('semantic-final-1')

## test on competition dataset

In [34]:
learn.load('semantic-final-1')

Learner(data=ImageDataBunch;

Train: LabelList (200 items)
x: SegmentationItemList
Image (3, 256, 512),Image (3, 256, 512),Image (3, 256, 512),Image (3, 256, 512),Image (3, 256, 512)
y: SegmentationLabelList
ImageSegment (1, 256, 512),ImageSegment (1, 256, 512),ImageSegment (1, 256, 512),ImageSegment (1, 256, 512),ImageSegment (1, 256, 512)
Path: data/minicity/leftImg8bit;

Valid: LabelList (100 items)
x: SegmentationItemList
Image (3, 256, 512),Image (3, 256, 512),Image (3, 256, 512),Image (3, 256, 512),Image (3, 256, 512)
y: SegmentationLabelList
ImageSegment (1, 256, 512),ImageSegment (1, 256, 512),ImageSegment (1, 256, 512),ImageSegment (1, 256, 512),ImageSegment (1, 256, 512)
Path: data/minicity/leftImg8bit;

Test: None, model=DynamicUnet(
  (layers): ModuleList(
    (0): Sequential(
      (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inpla

In [35]:
test_dataset = get_databunch(test_dir, size = (512,1024), bs =2)

In [36]:
learn.validate(test_dataset.valid_dl)

[1.5086988, tensor(0.6692), tensor(0.1136)]

## Preadict and save

In [41]:
save_to_eval(learn, test_dataset)    


In [43]:
!python toolkit/semantic-segmentation/evaluate.py --results results/ --minicity data/cityspaces/

Evaluating 500 pairs of images...
Images Processed: 500 

-------------- ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------- 
              |  u   |  e   |  r   |  o   |  s   |  d   |  g   |  r   |  s   |  p   |  r   |  b   |  w   |  f   |  g   |  b   |  t   |  p   |  p   |  t   |  t   |  v   |  t   |  s   |  p   |  r   |  c   |  t   |  b   |  c   |  t   |  t   |  m   |  b   | Prior |
-------------- ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------- 
    unlabeled | [31;1 0.00   [31;1 0.00   [31;1 0.00   [31;1 0.00   [31;1 0.00   [31;1 0.00   [31;1 0.00   [31;1 0.00   [31;1 0.00   [31;1 0.0